In [1]:
import os
import sys
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

from keras.layers import Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.losses import categorical_crossentropy

import tensorflow as tf

print(f'Python: {sys.version}')

Python: 3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]


In [2]:
RANDOM_STATE = 0

In [3]:
# Working with pre trained model 

#base_model = MobileNet(input_shape=(224, 224, 3), include_top=False)
base_model = MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)


for layer in base_model.layers:
  layer.trainable = False


x = Flatten()(base_model.output)
x = Dense(units=52 , activation='softmax')(x)

# creating our model.
model = Model(base_model.input, x)

In [4]:
model.compile(optimizer='adam', loss=categorical_crossentropy , metrics=['accuracy'])

In [ ]:
sessions = []
directory_template = './data/images/images/train/$session$/'
for session in os.listdir('./data/images/images/train'):
    # Find XML filename
    directory = directory_template.replace('$session$', session)
    files = os.listdir(directory)
    
    for file in files:
        # Load JPG filename
        jpg_filename = directory + file
        
        emotion_dict = {
            'emotion': session,
            'img': jpg_filename
        }
        sessions.append(emotion_dict)


In [ ]:
df_sessions = pd.DataFrame(sessions)
df_sessions.index.rename('session', inplace=True)
df_sessions["aus"] = 0
df_sessions.info

In [ ]:

df_aus = pd.read_excel('data/aus.xlsx')

In [ ]:
df_anger = df_sessions.loc[lambda df: df['emotion'] == "angry"].reset_index()
df_disgust = df_sessions.loc[lambda df: df['emotion'] == "disgust"].reset_index()
df_happy = df_sessions.loc[lambda df: df['emotion'] == "happy"].reset_index()
df_fear = df_sessions.loc[lambda df: df['emotion'] == "fear"].reset_index()
df_surprise = df_sessions.loc[lambda df: df['emotion'] == "surprise"].reset_index()
df_sadness = df_sessions.loc[lambda df: df['emotion'] == "sad"].reset_index()
df_neutral = df_sessions.loc[lambda df: df['emotion'] == "neutral"].reset_index()

In [ ]:
import random
list_anger = ["4,5,7,10,22,23,25,26", "4,5,7,10,23,25,27", "4,5,7,23", "4,5,7,17,23", "4,5,7,17,24"]
list_disgust = ["9,16,15,26", "9,17", "10, 17", "10,16,25,26"]
list_sadness = ["1,4,11,15,54,64","1,4,15","6,15,54,64","11,17","1,4,15,17"]
list_fear = ["1,2,4,5,20,25","5,20","1,2,4,5","1,2,4,5,25"]
list_surprise=["1,2,5,26", "5,27", "1,2,26", "5,26", "1,2,5", "1,2,5,27"]
list_happy = ["6,12"]
list_neutral = ["0"]

for i in range(len(df_anger.values)):
    rand_idx = random.randrange(len(list_anger))
    random_num = list_anger[rand_idx]

    df_anger["aus"][i] = random_num

i = 0
for i in range(len(df_disgust.values)):
    rand_idx = random.randrange(len(list_disgust))
    random_num = list_disgust[rand_idx]

    df_disgust["aus"][i] = random_num

i = 0
for i in range(len(df_sadness.values)):
    rand_idx = random.randrange(len(list_sadness))
    random_num = list_sadness[rand_idx]

    df_sadness["aus"][i] = random_num

i = 0
for i in range(len(df_fear.values)):
    rand_idx = random.randrange(len(list_fear))
    random_num = list_fear[rand_idx]

    df_fear["aus"][i] = random_num

i = 0
for i in range(len(df_surprise.values)):
    rand_idx = random.randrange(len(list_surprise))
    random_num = list_surprise[rand_idx]

    df_surprise["aus"][i] = random_num

df_happy["aus"] = list_happy[0]
df_neutral["aus" ] = list_neutral[0]

In [5]:
df_treino = pd.read_csv("base_aus.csv")
df_treino = df_treino.drop(columns="Unnamed: 0")
df_treino

,emotion,img,aus
0,angry,./data/images/images/train/angry/0.jpg,"4,5,7,10,22,23,25,26"
1,angry,./data/images/images/train/angry/1.jpg,"4,5,7,17,23"
2,angry,./data/images/images/train/angry/10.jpg,"4,5,7,23"
3,angry,./data/images/images/train/angry/10002.jpg,"4,5,7,10,23,25,27"
4,angry,./data/images/images/train/angry/10016.jpg,"4,5,7,17,23"
...,...,...,...
28816,neutral,./data/images/images/train/neutral/9987.jpg,0
28817,neutral,./data/images/images/train/neutral/999.jpg,0
28818,neutral,./data/images/images/train/neutral/9995.jpg,0
28819,neutral,./data/images/images/train/neutral/9998.jpg,0


In [6]:
train_datagen = ImageDataGenerator(
    zoom_range = 0.2, 
    shear_range = 0.2, 
    horizontal_flip=True, 
    rescale = 1./255
)
train_data = train_datagen.flow_from_dataframe(
    dataframe = pd.concat([df_treino]).sample(frac=1),
    directory=".",
    x_col = "img",
    y_col = "aus",
    subset = "training",
    batch_size = 32,
    seed = 42,
    class_mode = "categorical",
    target_size = (128,128)
)

Found 28821 validated image filenames belonging to 26 classes.


In [7]:
hist = model.fit(
    train_data,
    steps_per_epoch=10,
    epochs=30,
    validation_data=train_data,
    validation_steps=8,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            filepath='./results/model_aus.h5',
            save_best_only=True,
            monitor="accuracy"
        )
    ]
)

print()
max_acc = max(hist.history['accuracy']) * 100
print(f'Accuracy: {max_acc = :.2f}%')

Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "C:\Users\18.00050-9\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\18.00050-9\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\18.00050-9\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\18.00050-9\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "C:\Users\18.00050-9\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\18.00050-9\AppData\Local